In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from time import time
%matplotlib inline

#### Load data

In [2]:
# load csv files
train_df = pd.read_csv("train.csv")
profiles_df = pd.read_csv("profiles.csv")
artists_df = pd.read_csv("artists.csv")
artists_with_genres_df = pd.read_csv("artists_with_genres.csv")
user_median_df = pd.read_csv("user_median.csv")
global_median_df = pd.read_csv("global_median.csv")

#### Calculate mean ages per gender, country combination

In [3]:
# calculate mean age per user
mean_age = profiles_df[~profiles_df['age'].isnull()]["age"].mean()

###################################################################### 

# calculate mean age per men
males_only_df = profiles_df[profiles_df["sex"]=='m']
mean_male_age = males_only_df[~males_only_df['sex'].isnull()]["age"].mean()

###################################################################### 

# calculate mean age per women
females_only_df = profiles_df[profiles_df["sex"]=='f']
mean_female_age = females_only_df[~females_only_df['sex'].isnull()]["age"].mean()

###################################################################### 

# get list of unique countries
countries = profiles_df["country"].unique()

# calculate mean age per country
mean_country_age = {}
for country in countries:
    tmp_df = profiles_df[profiles_df["country"]==str(country)]
    mean_country_age[str(country)] = tmp_df["age"].mean()

###################################################################### 
    
# calculate mean age per male per country
mean_country_male_age = {}
for country in countries:
    tmp_df0 = profiles_df[profiles_df["sex"]=='m']
    tmp_df1 = tmp_df0[profiles_df["country"]==str(country)]
    mean_country_male_age[str(country)] = tmp_df1["age"].mean()

###################################################################### 

# calculate mean age per female per country
mean_country_female_age = {}
for country in countries:
    tmp_df0 = profiles_df[profiles_df["sex"]=='f']
    tmp_df1 = tmp_df0[profiles_df["country"]==str(country)]
    mean_country_female_age[str(country)] = tmp_df1["age"].mean()

//anaconda/lib/python2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


#### Create training dataframe with indicator variable columns and missing age values filled in

In [4]:
# initialize training df
training_df = profiles_df

###################################################################### 

# create sex indicators
training_df["male"] = 0
training_df["female"] = 0
training_df["sex_missing"] = 0
training_df.loc[training_df["sex"] =='m', "male"] = 1
training_df.loc[training_df["sex"] =='f', "female"] = 1
training_df.loc[training_df["sex"].isnull(), "sex_missing"] = 1

######################################################################  

# create age indicators
training_df["age_missing"] = 0
training_df.loc[training_df["age"].isnull(),"age_missing"] = 1

# fill in mean age: has no gender or country
training_df.loc[training_df["age_missing"] == 1, "age"] = mean_age

# fill in mean age: has gender, no country
#males
condition = (training_df["sex"] =='m') & (training_df["country"].isnull()) & (training_df["age_missing"] == 1)
training_df.loc[condition, "age"] = mean_male_age
# females
condition = (training_df["sex"] =='f') & (training_df["country"].isnull()) & (training_df["age_missing"] == 1)
training_df.loc[condition, "age"] = mean_female_age

# fill in mean age: has country, no gender
#loop over countries
for country in countries:
    condition = (training_df["country"] == country) & (training_df["age_missing"] == 1) & (training_df["sex_missing"] == 1)
    training_df.loc[condition, "age"] = mean_country_age[country]

# fill in mean age: has gender and country
# loop over countries
for country in countries:
    # males
    condition = (training_df["country"] == country) & (training_df["sex"] == 'm') & (training_df["age_missing"] == 1)
    training_df.loc[condition, "age"] = mean_country_male_age[country]
    # females
    condition = (training_df["country"] == country) & (training_df["sex"] == 'f') & (training_df["age_missing"] == 1)
    training_df.loc[condition, "age"] = mean_country_female_age[country]

###################################################################### 
    
# create country indicators
training_df["country_missing"] = 0
training_df.loc[training_df["country"].isnull(), "country_missing"] = 1
# loop over countries
for country in countries:
    # assign 1 if in that country
    training_df.loc[training_df["country"] == country, country] = 1
    # assign 0 otherwise
    training_df.loc[training_df["country"] != country, country] = 0

#### Transfer plays per artist data from train df to our training df. one column per artist

In [5]:
## Initialize all artist columns to 0
artists = train_df["artist"].unique()
for artist in artists:
    training_df[artist]=0
 
######################################################################

# create helper function to keep track of loop progress and est. time remaining
def progress(step, steps, t0):
    for i in range(1,10):
        if (step == (steps/10)*i):
            percent_complete = int(100*((steps/10)*i)/steps)
            seconds_remaining = int(((time()-t0)/i)*(10-i))
            print "\r" + str(percent_complete) + "% complete " + str(seconds_remaining) + " seconds remaining",

######################################################################

# set index to user for faster computation
training_df = training_df.set_index("user")
counter = 0
t0 = time()

#########################
# WARNING: TAKES ~80Min #
#########################

# loop through each row in train_df and plug play value into training_df
for index, row in train_df.set_index("user").iterrows():
    training_df.loc[index, row[0]] = row[1] #index = user id, row[0] = artist id, row[1] = plays
    counter += 1
    progress(counter, len(train_df), t0) # print progress (every 10ppts)

89% complete 354 seconds remaining


In [6]:
training_df.to_csv("training_df.csv")

In [8]:
training_df.head(400)

,sex,age,country,male,female,sex_missing,age_missing,country_missing,Sweden,Iceland,...,e390d54f-92f0-4039-9dd3-e8eac130b4c0,72d7d717-0837-4f2a-9641-d0f9fdd3acf7,6ae51665-8261-4ae5-883f-1899651ad31b,ca264abf-3eb6-4d53-827b-6ab16988a4a3,c78a5d46-c631-41bc-8772-439002e3d3aa,dd3f655b-7bf6-4fe1-9360-3de5d842e9d2,582fee78-d934-4603-9f5c-1729e0ca36b7,c1d4f2ba-cf39-460c-9528-6b827d3417a1,bfcc6d75-a6a5-4bc6-8282-47aec8531818,774666d2-2064-4d6c-856c-f8cda0aaf9f0
user,,,,,,,,,,,,,,,,,,,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25.000000,Sweden,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5909125332c108365a26ccf0ee62636eee08215c,m,29.000000,Iceland,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30.000000,United States,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63268cce0d68127729890c1691f62d5be5abd87c,m,21.000000,Germany,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02871cd952d607ba69b64e2e107773012c708113,m,24.000000,Netherlands,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0938eb3d1b449b480c4e2431c457f6ead7063a34,m,22.000000,United States,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e4c6b36e65db3d48474dd538fe74d2dbb5a2e79e,f,23.511588,United States,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b97479f9a563a5c43b423a976f51fd509e1ec5ba,f,21.213703,Poland,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3bb020df0ff376dfdded4d5e63e2d35a50b3c535,m,26.152497,United States,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
